In [1]:
train = pd.read_csv("data/train.csv")
train.tail()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0
159570,fff46fc426af1f9a,"""\nAnd ... I really don't think you understand...",0,0,0,0,0,0


In [2]:
test = pd.read_csv("data/test.csv")

In [7]:
train.loc[159566,'comment_text']

'":::::And for the second time of asking, when your view completely contradicts the coverage in reliable sources, why should anyone care what you feel? You can\'t even give a consistent argument - is the opening only supposed to mention significant aspects, or the ""most significant"" ones?   \n\n"'

### Data cleaning

In [3]:
import re
def clean_comment(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    
    return text

In [16]:
print(train['comment_text'][0],'\n\n',clean_comment(train['comment_text'][0]))

Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27 

 explan whi the edit made under my usernam hardcor metallica fan were revert they were not vandal just closur on some ga after i vote at new york doll fac and pleas do not remov the templat from the talk page sinc i am retir now 89 205 38 27


In [4]:
train['clean_comment'] = train['comment_text'].map(lambda comment:clean_comment(comment))
test['clean_comment'] = test['comment_text'].map(lambda comment:clean_comment(comment))

In [5]:
train.tail()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean_comment
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0,and for the second time of asking when your vi...
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0,you should be ashamed of yourself that is a ho...
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0,spitzer umm theres no actual article for prost...
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0,and it looks like it was actually you who put ...
159570,fff46fc426af1f9a,"""\nAnd ... I really don't think you understand...",0,0,0,0,0,0,and i really do not think you understand i cam...


In [19]:
x_train = train.clean_comment
x_test = test.clean_comment

### Vectorize

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')

In [22]:
x_train_dtm = vectorizer.fit_transform(x_train)
x_test_dtm = vectorizer.fit_transform(x_test)

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

logreg = LogisticRegression(C=12.0)
cols_target = ['obscene','insult','toxic','severe_toxic','identity_hate','threat']

submission_binary = pd.read_csv('data/sample_submission.csv')

for label in cols_target:
    print('... Processing {}'.format(label))
    y = train[label]
    # train the model using X_dtm & y
    logreg.fit(x_train_dtm, y)
    # compute the training accuracy
    y_pred_X = logreg.predict(x_train_dtm)
    print('Training accuracy is {}'.format(accuracy_score(y, y_pred_X)))
    # compute the predicted probabilities for X_test_dtm
    test_y_prob = logreg.predict_proba(x_test_dtm)[:,1]
    submission_binary[label] = test_y_prob

... Processing obscene
Training accuracy is 0.9832300355327722
... Processing insult
Training accuracy is 0.9755469352200588
... Processing toxic
Training accuracy is 0.9639846839337975
... Processing severe_toxic
Training accuracy is 0.9920724943755445
... Processing identity_hate
Training accuracy is 0.9939713356436947
... Processing threat
Training accuracy is 0.9981199591404453


In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, KFold

In [12]:
# forest = RandomForestClassifier(n_estimators=100, n_jobs=3, random_state=0)
# cols_target = ['obscene','insult','toxic','severe_toxic','identity_hate','threat']

# submission_binary = pd.read_csv('data/sample_submission.csv')

# for label in cols_target:
#     print('... Processing {}'.format(label))
#     y = train[label]
#     x_train, x_test, y_train, y_test = train_test_split(x_train_dtm, y, test_size=0.33, random_state=0)
#     model = forest.fit(x_train, y_train)
#     y_pred_train = model.predict(x_test)
#     print("Accuracy score :", accuracy_score(y_test, y_pred_train))
#     test_y_prob = model.predict_proba(x_test_dtm)[:,1]
#     submission_binary[label] = test_y_prob

... Processing obscene
Accuracy score : 0.9791108832298373
... Processing insult
Accuracy score : 0.9688942061186122
... Processing toxic
Accuracy score : 0.9538920222564045
... Processing severe_toxic
Accuracy score : 0.9896124119333827
... Processing identity_hate
Accuracy score : 0.9913405115934598
... Processing threat
Accuracy score : 0.9971894642891054


In [24]:
submission_binary.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.008964,0.003194,0.005807,0.000744,0.005809,0.000414
1,0000247867823ef7,0.005399,0.012859,0.002200,0.000310,0.006573,0.001359
2,00013b17ad220c46,0.016057,0.001256,0.001958,0.000335,0.001432,0.000697
3,00017563c3f7919a,0.028739,0.001306,0.019503,0.000378,0.006219,0.000856
4,00017695ad8997eb,0.174040,0.002908,0.017808,0.001148,0.025765,0.001674


In [25]:
submission_binary.to_csv('submission_binary_forest.csv',index=False)

In [27]:
# create submission file
submission_chains = pd.read_csv('data/sample_submission.csv')

# create a function to add features
def add_feature(X, feature_to_add):
    '''
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    '''
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

In [28]:
for label in cols_target:
    print('... Processing {}'.format(label))
    y = train[label]
    # train the model using X_dtm & y
    logreg.fit(x_train_dtm,y)
    # compute the training accuracy
    y_pred_X = logreg.predict(x_train_dtm)
    print('Training Accuracy is {}'.format(accuracy_score(y,y_pred_X)))
    # make predictions from test_X
    test_y = logreg.predict(x_test_dtm)
    test_y_prob = logreg.predict_proba(x_test_dtm)[:,1]
    submission_chains[label] = test_y_prob
    # chain current label to X_dtm
    x_train_dtm = add_feature(x_train_dtm, y)
    print('Shape of X_dtm is now {}'.format(x_train_dtm.shape))
    # chain current label predictions to test_X_dtm
    x_test_dtm = add_feature(x_test_dtm, test_y)
    print('Shape of test_X_dtm is now {}'.format(x_test_dtm.shape))

... Processing obscene
Training Accuracy is 0.9832300355327722
Shape of X_dtm is now (159571, 5001)
Shape of test_X_dtm is now (153164, 5001)
... Processing insult
Training Accuracy is 0.9819202737339492
Shape of X_dtm is now (159571, 5002)
Shape of test_X_dtm is now (153164, 5002)
... Processing toxic
Training Accuracy is 0.9676194295956032
Shape of X_dtm is now (159571, 5003)
Shape of test_X_dtm is now (153164, 5003)
... Processing severe_toxic
Training Accuracy is 0.9931065168482995
Shape of X_dtm is now (159571, 5004)
Shape of test_X_dtm is now (153164, 5004)
... Processing identity_hate
Training Accuracy is 0.9955819039800465
Shape of X_dtm is now (159571, 5005)
Shape of test_X_dtm is now (153164, 5005)
... Processing threat
Training Accuracy is 0.9985774357496037
Shape of X_dtm is now (159571, 5006)
Shape of test_X_dtm is now (153164, 5006)


In [29]:
submission_chains.to_csv('submission_chains.csv', index=False)

In [30]:
submission_combined = pd.read_csv('data/sample_submission.csv')

In [31]:
for label in cols_target:
    submission_combined[label] = 0.5*(submission_chains[label]+submission_binary[label])

In [32]:
submission_combined.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.006041,0.001609,0.005807,0.000474,0.003746,0.000247
1,0000247867823ef7,0.004819,0.006496,0.002200,0.000193,0.006405,0.000784
2,00013b17ad220c46,0.018889,0.000631,0.001958,0.000200,0.001719,0.000445
3,00017563c3f7919a,0.024260,0.000655,0.019503,0.000220,0.004953,0.000482
4,00017695ad8997eb,0.150842,0.001460,0.017808,0.000660,0.016205,0.000923


In [33]:
submission_combined.to_csv('submission_combined.csv', index=False)